In [1]:
import anndata
import pandas as pd

In [2]:
path = "../07_eqtls/scored_sceqtl_vars.h5ad"

In [ ]:
ad = anndata.read_h5ad(path)

In [4]:
ad

AnnData object with n_obs × n_vars = 18171 × 8856
    obs: 'gene_id', 'gene_symbol', 'variant', 'rsid', 'chrom', 'pos', 'ref', 'alt', 'gene_strand', 'pos_relative', 'index'
    var: 'cell_type', 'tissue', 'organ', 'disease', 'study', 'dataset', 'region', 'subregion', 'celltype_coarse', 'n_cells', 'total_counts', 'n_genes', 'size_factor', 'train_pearson', 'val_pearson', 'test_pearson'
    layers: 'ref_expr'

In [21]:
ad.obs.head()

,gene_id,gene_symbol,variant,rsid,chrom,pos,ref,alt,gene_strand,pos_relative,index
980,ENSG00000170430,MGMT,chr10_129467281_C_T,rs16906252,chr10,129467281,C,T,+,163930,980
1013,ENSG00000180817,PPA1,chr10_70233338_T_G,rs7077538,chr10,70233338,A,C,-,164413,1013
1063,ENSG00000225484,NUTM2B-AS1,chr10_79758006_G_A,rs7921005,chr10,79758006,C,T,-,232428,1063
1262,ENSG00000134809,TIMM10,chr11_57516515_C_T,rs2848626,chr11,57516515,G,A,-,178128,1262
1640,ENSG00000183020,AP2A2,chr11_973542_C_T,rs10902256,chr11,973542,C,T,+,212487,1640


In [8]:
ad.var.head()

,cell_type,tissue,organ,disease,study,dataset,region,subregion,celltype_coarse,n_cells,total_counts,n_genes,size_factor,train_pearson,val_pearson,test_pearson
agg_0,Amygdala excitatory,Amygdala_Amygdala,CNS,healthy,jhpce#tran2021,brain_atlas,Amygdala,Amygdala,NaN,331,15928830.0,17000,41431.465186,0.942474,0.841879,0.865579
agg_1,Amygdala excitatory,Amygdala_Basolateral nuclear group (BLN) - lat...,CNS,healthy,SCR_016152,brain_atlas,Amygdala,Basolateral nuclear group (BLN) - lateral nucl...,NaN,11369,295213300.0,18080,40765.341481,0.943247,0.839565,0.860787
agg_2,Amygdala excitatory,Amygdala_Bed nucleus of stria terminalis and n...,CNS,healthy,SCR_016152,brain_atlas,Amygdala,Bed nucleus of stria terminalis and nearby - BNST,NaN,139,2593231.0,15418,42556.387020,0.952271,0.855175,0.866643
agg_3,Amygdala excitatory,Amygdala_Central nuclear group - CEN,CNS,healthy,SCR_016152,brain_atlas,Amygdala,Central nuclear group - CEN,NaN,3892,99463712.0,17959,42884.641430,0.959857,0.863817,0.881534
agg_4,Amygdala excitatory,Amygdala_Corticomedial nuclear group (CMN) - a...,CNS,healthy,SCR_016152,brain_atlas,Amygdala,Corticomedial nuclear group (CMN) - anterior c...,NaN,2945,128161939.0,17885,41816.741933,0.951541,0.855172,0.868277


In [12]:
susie = pd.read_csv("../07_eqtls/scored_sceqtls.csv", index_col=0)

In [17]:
pos_vg_pairs = susie[susie.label][['gene_id', 'variant']].drop_duplicates()

In [25]:
pos_vg_indices = ad.obs[['gene_id', 'variant', 'index']].merge(pos_vg_pairs, how="inner")['index'].tolist()

In [27]:
ad.obs['negative_control'] = True

In [29]:
ad.obs.loc[ad.obs['index'].isin(pos_vg_indices), 'negative_control'] = False

In [30]:
del ad.obs['index']

In [31]:
ad.obs.negative_control.value_counts()

negative_control
True     17598
False      573
Name: count, dtype: int64

In [32]:
ad

AnnData object with n_obs × n_vars = 18171 × 8856
    obs: 'gene_id', 'gene_symbol', 'variant', 'rsid', 'chrom', 'pos', 'ref', 'alt', 'gene_strand', 'pos_relative', 'negative_control'
    var: 'cell_type', 'tissue', 'organ', 'disease', 'study', 'dataset', 'region', 'subregion', 'celltype_coarse', 'n_cells', 'total_counts', 'n_genes', 'size_factor', 'train_pearson', 'val_pearson', 'test_pearson'
    layers: 'ref_expr'

In [33]:
del ad.layers['ref_expr']

In [34]:
ad.X

array([[-4.79987860e-02, -2.56800652e-02, -2.29399800e-02, ...,
        -5.49203157e-02, -8.26480389e-02, -5.03803492e-02],
       [ 6.47183061e-02,  6.56886101e-02,  4.75059152e-02, ...,
         1.01768196e-01,  9.23620462e-02,  1.04189277e-01],
       [-1.39474869e-05, -9.38177109e-05,  3.75509262e-05, ...,
         1.61767006e-04, -2.54124403e-04, -1.23023987e-04],
       ...,
       [-3.78966331e-04, -3.48210335e-04,  3.00407410e-05, ...,
        -1.10507011e-04,  1.46937370e-03,  1.61629915e-03],
       [ 4.09531593e-03,  2.95615196e-03, -1.41167641e-03, ...,
         4.64391708e-03, -1.81382895e-03, -6.98685646e-04],
       [ 1.59502029e-04, -1.02519989e-05,  1.13725662e-04, ...,
         2.80618668e-04,  5.22732735e-05, -2.38418579e-06]], dtype=float32)

In [35]:
ad

AnnData object with n_obs × n_vars = 18171 × 8856
    obs: 'gene_id', 'gene_symbol', 'variant', 'rsid', 'chrom', 'pos', 'ref', 'alt', 'gene_strand', 'pos_relative', 'negative_control'
    var: 'cell_type', 'tissue', 'organ', 'disease', 'study', 'dataset', 'region', 'subregion', 'celltype_coarse', 'n_cells', 'total_counts', 'n_genes', 'size_factor', 'train_pearson', 'val_pearson', 'test_pearson'

In [36]:
ad.write_h5ad("Data_File_2_sceqtl_preds.h5ad")